In [1]:
from transformers import BertTokenizer, GPT2LMHeadModel
import torch
from tqdm import trange
import os
import numpy as np
import random
import time

In [2]:
def seed_everything(seed: int = 42):
    """Util to make training reproducible"""
    random.seed(seed)

    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    if os.getenv("CUBLAS_WORKSPACE_CONFIG") is not None:
        torch.use_deterministic_algorithms(True)
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
seed_everything(1999)
tok_path = '..\\..\\Raw_GPT2\\vocab.txt'
pretrain_model_path = "D:\\VSCode\\Record_By_Date\\2023_02_20\\model\\best_model\\"
# output_dir = "model\\"

tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(pretrain_model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print('using device:', device)
model = model.to(device)
model.eval()

using device: cpu


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [4]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float("Inf")):
    """Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
    Args:
        logits: logits distribution shape (vocabulary size)
        top_k > 0: keep only top k tokens with highest probability (top-k filtering).
        top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
            Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert (
        logits.dim() == 1
    )  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(torch.nn.functional.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

def sample_sequence_batch(
    model,
    context,
    length,
    n_ctx,
    tokenizer,
    temperature=1.0,
    top_k=30,
    top_p=0.0,
    repetition_penalty=1.0,
    device="cpu",
):
    context = torch.tensor(context, dtype=torch.long, device=model.device)
    generated = context

    with torch.no_grad():
        for _ in trange(length):
            inputs = {"input_ids": generated[:, -(n_ctx - 1) :]}
            outputs = model(**inputs)
            next_token_logits = outputs[0][:, -1, :]

            for idx, gen in enumerate(generated):
                for id in set(gen):
                    next_token_logits[idx, id] /= repetition_penalty

            next_token_logits = next_token_logits / temperature
            next_token_logits[:, tokenizer.convert_tokens_to_ids("[UNK]")] = -float("Inf")
            filtered_logits = torch.stack([top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p) for logits in next_token_logits])

            next_tokens = torch.multinomial(torch.nn.functional.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_tokens), dim=1)
    
    return generated.tolist()

def is_word(word):
    for item in list(word):
        if item not in "qwertyuiopasdfghjklzxcvbnm":
            return False
    return True

In [6]:
raw_text = "前列腺炎的症状都有哪些？"
# 10, 5
nsamples = 100
batch_size = 20
length = 30
temperature = 1.0
repitition_penalty = 5
top_k = 40
top_p = 0
n_ctx = 1024

save_samples = True

if save_samples:
    # if not os.path.exists(save_samples_path):
    #     os.makedirs(save_samples_path)
    samples_file = open("samples.txt", "w", encoding="utf8")

while True:
    
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(raw_text))
    generated = 0
    for _ in range(nsamples // batch_size):
        # seed_everything(int(time.time() * 1000) % (2**32 - 1))
        context_tokens_batch = [context_tokens for _ in range(batch_size)]
        out = sample_sequence_batch(
            model,
            context_tokens_batch,
            length,
            n_ctx,
            tokenizer=tokenizer,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repitition_penalty,
            device=device,
        )
        
        for i in range(batch_size):
            generated += 1
            
            text = tokenizer.convert_ids_to_tokens(torch.tensor(out[i]))
            for it, item in enumerate(text[:-1]):  # 确保英文前后有空格
                if is_word(item) and is_word(text[it + 1]):
                    text[it] = item + " "
            for it, item in enumerate(text):
                if item == "[MASK]":
                    text[it] = ""
                elif item == "\n":
                    text[it] = ""
                elif item == "[CLS]":
                    text[it] = "\n\n"
                elif item == "[SEP]":
                    text[it] = "\n"
            info = "=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40 + "\n"
            print(info)
            text = "".join(text).replace("##", "").strip()
            print(text)
            if save_samples:
                samples_file.write(info)
                samples_file.write(text)
                samples_file.write("\n")
                samples_file.write("=" * 90)
                samples_file.write("\n" * 2)
    print("=" * 80)
    
    break
samples_file.close()

100%|██████████| 30/30 [00:30<00:00,  1.01s/it]


======================================== SAMPLE 1 ========================================

前列腺炎的症状都有哪些？

我今年25岁，从上个多月开始发觉已经是排位赛了。这几天又犯病
======================================== SAMPLE 2 ========================================

前列腺炎的症状都有哪些？尿频、小腹胀痛，甚至下坠感。您好：导致男性睾丸疼应该及时检查
======================================== SAMPLE 3 ========================================

前列腺炎的症状都有哪些？1、久坐：常会头晕眼花，大便干燥或疼痛。

男性发病率较女士减
======================================== SAMPLE 4 ========================================

前列腺炎的症状都有哪些？

一定要及时治疗，根据你上传到咨询室里面是关键所在。只不过仔
======================================== SAMPLE 5 ========================================

前列腺炎的症状都有哪些？

如果再次出现了泌尿生殖系疾病会引发肾脏不适，建议去正规医院
======================================== SAMPLE 6 ========================================

前列腺炎的症状都有哪些？您好：

性交引来阴茎勃起硬度不够，而且还会时间过长或者一直持
======================================== SAMPLE 7 ========================================

前列腺炎的症状都有哪些？我最近好象是在早晨上厕所发现尿血，下班就没太注意过这个问题。
======================================== SAMPLE 8

100%|██████████| 30/30 [00:30<00:00,  1.00s/it]


======================================== SAMPLE 21 ========================================

前列腺炎的症状都有哪些？患了慢性细菌感染，又该怎么治疗呢?1、可能会出现排尿异常。此
======================================== SAMPLE 22 ========================================

前列腺炎的症状都有哪些？男性疾病和长期憋尿是两种不同人类精液生化学物质沉积结构中必需
======================================== SAMPLE 23 ========================================

前列腺炎的症状都有哪些？

怎么才能更好地预防和治疗工作这种急性男科疾病呢?您还年轻;
======================================== SAMPLE 24 ========================================

前列腺炎的症状都有哪些？?

不要长久地看病。男人得了阳痿、早泄等疾会致使精神无法集中
======================================== SAMPLE 25 ========================================

前列腺炎的症状都有哪些？治疗方法是什么呢?主要应该多喝水,一般可以服用药物，抗生素、
======================================== SAMPLE 26 ========================================

前列腺炎的症状都有哪些？你好，可以进行一下分类：

看具体情况。我建议选择阴囊皮肤和龟
======================================== SAMPLE 27 ========================================

前列腺炎的症状都有哪些？

一般是因为长期久坐或者着凉引发上述临床综合征，多位于中老年
======================================== SAM

100%|██████████| 30/30 [00:29<00:00,  1.00it/s]


======================================== SAMPLE 41 ========================================

前列腺炎的症状都有哪些？1.尿频。

一天2-3次或两三个小时以上偶尔还伴发排泄物，大
======================================== SAMPLE 42 ========================================

前列腺炎的症状都有哪些？慢性男科疾病引来，常见于无菌生理卫

不排除泌尿系感染情况致使
======================================== SAMPLE 43 ========================================

前列腺炎的症状都有哪些？

在患病期间应该注意什么呢?怎样治疗才能比较好啊你是需要做个
======================================== SAMPLE 44 ========================================

前列腺炎的症状都有哪些？

一般来说，除了上述三种表现出外之后不会再增加任何痛苦。也就
======================================== SAMPLE 45 ========================================

前列腺炎的症状都有哪些？男性出现了泌尿系统感染，或者是精神上受到刺激后会再次发生排便
======================================== SAMPLE 46 ========================================

前列腺炎的症状都有哪些？

尿急，排不出来、腰酸背痛等！一般是因为患了盆腔感染引起白带
======================================== SAMPLE 47 ========================================

前列腺炎的症状都有哪些？你好，慢性浅表精囊液增长会出现下腹痛或小便不畅,也就是说排尿
======================================== SA

100%|██████████| 30/30 [00:29<00:00,  1.01it/s]


======================================== SAMPLE 61 ========================================

前列腺炎的症状都有哪些？一年多了没任何征兆！每次月经后不久即射精，排尿时再也无法控制
======================================== SAMPLE 62 ========================================

前列腺炎的症状都有哪些？你好，治疗可以选择手术

如内服药、微波,热水坐浴等.同时注意
======================================== SAMPLE 63 ========================================

前列腺炎的症状都有哪些？什么时候患上比较好呢?

大概多久才能检测出来啊你可以去医院确
======================================== SAMPLE 64 ========================================

前列腺炎的症状都有哪些？男性阴囊内出现了许多小块，而且尿液中还含蛋白质、脂肪不仅干扰
======================================== SAMPLE 65 ========================================

前列腺炎的症状都有哪些？?尿频(比如:上厕所时要特别小心，这是一种很简单而容易困扰男
======================================== SAMPLE 66 ========================================

前列腺炎的症状都有哪些？最近总是感觉尿频，并且每次排便完以后还会出血情况吗。请问：得
======================================== SAMPLE 67 ========================================

前列腺炎的症状都有哪些？会不定时地疼痛吗，是一个怎样子感觉呢除了在饮食上要注意什么您
======================================== SAMPL

100%|██████████| 30/30 [00:29<00:00,  1.00it/s]

======================================== SAMPLE 81 ========================================

前列腺炎的症状都有哪些？

请问治疗男性不育那种方法好呢您这两个病应该到正规医院仔细检
======================================== SAMPLE 82 ========================================

前列腺炎的症状都有哪些？

因为不清楚，在这里先咨询一下!你好：治疗上首选解痉药物和镇
======================================== SAMPLE 83 ========================================

前列腺炎的症状都有哪些？1、性功能障碍。

女孩子在月经干净后,阴道会轻度不规则疼;而
======================================== SAMPLE 84 ========================================

前列腺炎的症状都有哪些？你好，这种情况应该和性交过度关系没那么大。

如果担心会影响生
======================================== SAMPLE 85 ========================================

前列腺炎的症状都有哪些？

对于我来说，好烦!。一般患了这个病之后会比较难过自己是不孕
======================================== SAMPLE 86 ========================================

前列腺炎的症状都有哪些？

病情分析：您好，阴茎不能勃起来或无力、性生活时间过久是男子
======================================== SAMPLE 87 ========================================

前列腺炎的症状都有哪些？

在性生活时阴茎硬度和血压是否正常，尿量应该如何判断。男人患
======================================== 